# MSST-Webui For Google Colab

LICENSE: AGPL-3.0

仅限个人娱乐和非商业用途。禁止用于涉及血腥、暴力、性或政治内容的使用。<br> 
Author: [Github@KitsuneX07](https://github.com/KitsuneX07) | [Github@SUC-DriverOld](https://github.com/SUC-DriverOld) | Gradio theme: [Gradio Theme](https://huggingface.co/spaces/NoCrypt/miku)<br>
Github: [https://github.com/SUC-DriverOld/MSST-WebUI](https://github.com/SUC-DriverOld/MSST-WebUI)

## 介绍/Introduction

这是一个用于音乐源分离训练[Music-Source-Separation-Training (MSST)](https://github.com/ZFTurbo/Music-Source-Separation-Training)的网页界面（webUI）,  MSST 是一个用于训练音乐源分离模型的代码仓库。你可以使用这个网页界面来推理 MSST 模型和 VR 模型，预设处理页面允许你自定义处理流程。你可以在“安装模型”界面中安装模型。如果你之前已经下载过 [Ultimate Vocal Remover (UVR)](https://github.com/Anjok07/ultimatevocalremovergui) , 则无需再次下载 VR 模型。你可以直接在“设置”页面选择你的 UVR5 模型文件夹。我们还在网页界面中提供了一些方便的工具，例如面向歌声的 MIDI 提取器 [Singing-Oriented MIDI Extractor (SOME)](https://github.com/openvpi/SOME/)、高级集成模式等功能
 1. 更改运行时类型，点击右上角的下拉箭头，选择更改运行时类型
在弹出来的页面选择Python3，T4 GPU（付费用户可以用A100和L4），在弹出来的对话框中选择“确定”，然后保存。等待其重启运行时，直到右上角显示“√ 已连接”。
2. 点击左侧竖着的侧栏中的文件夹图标，打开项目文件夹。Colab运行中的所有文件会存在这里。
3. 右键某文件夹可以上传本地电脑中的文件到colab中
4. 点击文件后面的三个点，可以选择下载文件。注意是文件，不是文件夹！



In [ ]:
# @title 连接到 Colab 运行时并检查 GPU/Connect to colab runtime and check GPU
# @markdown # 连接到 Colab 运行时并检查 GPU
# @markdown # Connect to colab runtime and check GPU
# @markdown

!nvidia-smi

In [ ]:
# @title 安装 python 3.10, ffmpeg/Install python 3.10, ffmpeg
# @markdown # 安装 python 3.10, ffmpeg
# @markdown
# @markdown # Install python 3.10, ffmpeg
# @markdown

!sudo apt install python3.10
!sudo apt install ffmpeg

In [ ]:
# @title 克隆仓库并安装依赖/Clone repository and install requirements
# @markdown # 克隆仓库并安装依赖/Clone repository and install requirements
# @markdown
# @markdown ### 你可能需要重新运行这个单元格几次
# @markdown ### You may need to rerun this cell for a few times
# @markdown ### 每次都点击重启会话，然后等待其重启完成。即右上角显示“√ 已连接”。
# @markdown ### 每次重启完成后需要重新运行该代码框。算上初次运行，一共要重复运行2-3次此代码框。直到最后一次运行后，运行按钮变成“√”为止。
# @markdown

# Clone the repository
%cd /content
!git clone https://github.com/SUC-DriverOld/MSST-WebUI
%cd /content/MSST-WebUI
!git checkout ccbce58

# Upgrade pip and setuptools
%pip install --upgrade pip setuptools

# Install requirements
%pip install tools/webUI_for_clouds/librosa-0.9.2-py3-none-any.whl
%pip install -r requirements.txt --only-binary=samplerate

In [ ]:
# @title 【可选】如果需要可下载SOME权重模型/[Optional] Download SOME weights if needed
# @markdown # 如果需要可下载SOME权重模型/Download SOME weights if needed
# @markdown
# @markdown ### 如果你要使用小工具中的歌声转MIDI，需要运行此代码框
# @markdown ### If you want to use "Vocals to MIDI", download the weights.
# @markdown ### "Vocals to MIDI" use project: [SOME](https://github.com/openvpi/SOME/).
# @markdown

!wget https://huggingface.co/Sucial/MSST-WebUI/resolve/main/SOME_weights/model_steps_64000_simplified.ckpt -O /content/MSST-WebUI/tools/SOME_weights/model_steps_64000_simplified.ckpt

In [ ]:
# @title 【可选】从你的Google Drive里上传需要处理的音频/[Optional] Pack and upload your music files to your google drive
# @markdown # 从你的Google Drive里上传需要处理的音频/Pack and upload your music files to your google drive# @markdown
# @markdown 确保你的压缩包里没有嵌套文件夹，而是直接播放音频文件# @markdown
# @markdown Ensure that there are no nested folders in your zip file, but rather direct audio files.# @markdown
# @markdown 针对多条音频，单个音频处理可以直接在webui上传# @markdown
# @markdown 初次运行此代码框时，会弹出窗口。点击“连接到Google云端硬盘”，然后根据提示一步一步使用谷歌账号登录，关联你的Google Drive。等待其重启运行时，直到右上角显示“√ 已连接”。完成此步后，在左侧文件夹内会看到你的Google Drive被映射进来了  
# @markdown

from google.colab import drive

drive.mount("/content/drive")
# @markdown 包含 **你的 zip 文件** 的目录，不要忘记末尾的斜杠  
# @markdown
# @markdown Directory where **your zip file** located in, dont miss the slash at the end.
data_dir = "/content/drive/MyDrive/MSST-WebUI/"  # @param {type:"string"}
# @markdown **你的压缩文件**的文件名，例如：“inputs.zip”
# @markdown
# @markdown Filename of **your zip file**, for example: "inputs.zip"
zip_filename = "input.zip"  # @param {type:"string"}
ZIP_PATH = data_dir + zip_filename
# @markdown 你需要将音频直接打包到一个 zip 文件中，**不要**在 zip 文件中嵌套文件夹！  
# @markdown You need to package the audio directly into a zip file, **DO NOT** nest folders in the zip file!  
# @markdown 压缩文件将被解压到 `input` 目录。如果你的 `input` 目录中已有文件，它们将被删除。请提前备份  
# @markdown The compressed file will be extracted to the `input` directory. If there are files in your `input` directory, they will be deleted. Please backup in advance.  
# @markdown

!rm -rf /content/MSST-WebUI/input
!mkdir -p /content/MSST-WebUI/input
!unzip -od /content/MSST-WebUI/input {ZIP_PATH}

In [ ]:
# @title 【可选】将WebUI处理后的结果文件保存到Google Drive/[Optional] Save results to google drive
# @markdown # 将WebUI处理后的结果文件保存到Google Drive/Save results to google drive  
# @markdown 输出目录中的结果音频文件将被压缩成一个 zip 文件并保存到你的 Google 云端硬盘
# @markdown The results audio files in the output directory will be compressed into a zip file and saved to your google drive.
# @markdown

from google.colab import drive

drive.mount("/content/drive")
# @markdown 保存 zip 文件的路径，末尾不要漏掉斜杠
# @markdown
# @markdown Path to save the zip file, dont miss the slash at the end.
output_dir = "/content/drive/MyDrive/MSST-WebUI/"  # @param {type:"string"}
# @markdown
# @markdown 压缩文件的文件名，例如：“Outputs.zip”
# @markdown
# @markdown Filename of the zip file, for example: "Outputs.zip"
zip_filename = "Outputs.zip"  # @param {type:"string"}
ZIP_PATH = output_dir + zip_filename

!mkdir -p {output_dir}
!zip -r "{zip_filename}" /content/MSST-WebUI/results
!cp -vr "{zip_filename}" {output_dir}

In [ ]:
# @title 【可选】一键删除results文件夹内的内容/[Optional] Delete all result audio files
# @markdown # 一键删除results文件夹内的内容/Click to delete all result audio files
# @markdown ### 警告：不可恢复，请谨慎操作
# @markdown ### Warning: Unrecoverable, please operate with caution
# @markdown

!rm -rf /content/MSST-WebUI/results
!mkdir -p /content/MSST-WebUI/results

In [ ]:
# @title 【可选】一键删除input文件夹内的内容/[Optional] Delete all input audio files
# @markdown # 一键删除input文件夹内的内容/Click to delete all input audio files
# @markdown ### 警告：不可恢复，请谨慎操作
# @markdown ### Warning: Unrecoverable, please operate with caution
# @markdown

!rm -rf /content/MSST-WebUI/input
!mkdir -p /content/MSST-WebUI/input

In [ ]:
# @title 初始化并启动 WebUI/Initialize and start WebUI
# @markdown # 初始化并启动 WebUI/Initialize and start WebUI
# @markdown 运行此单元格后，您可以使用自己的设备连接到 WebUI。公共链接将显示在下面
# @markdown After running this cell, you can use your own device to connect to WebUI. The public link will be displayed below.
# @markdown 如果你想修改预设流程的推理参数，请编辑 `data/webui_config.json` 文件
# @markdown If you want to modify the inference parameters of the preset process, edit the `data/webui_config.json` file.
# @markdown 最后一步，选择WebUI使用的语言及其他设置（建议仅修改语言，别的保存默认）选择完毕后运行代码框，即可启动WebUI
# @markdown 启动完毕后，下方输出栏会输出几个链接，其中Running on public URL是公网链接，是给你来连接WebUI的，直接点击即可跳转至WebUI（加载的会有点慢，如果出不来可以疯狂刷新网页）。如下图，点击框出来的链接！
# @markdown 在使用过程中可能会出现丢失连接。这时，请你重新运行colab中的最后一个启动WebUI的代码框，然后从新的公网链接重新进入
# @markdown

%cd /content/MSST-WebUI

LANGUAGE = "English"  # @param ["Auto", "English", "简体中文", "繁體中文", "日本語", "😊", "한국어"]
MODEL_DOWNLOAD_LINK = "huggingface.co"  # @param ["Auto", "huggingface.co", "hf-mirror.com"]
DEBUG = False  # @param {type:"boolean"}

language_dict = {"Auto": "Auto", "简体中文": "zh_CN", "繁體中文": "zh_TW", "English": "en_US", "日本語": "ja_JP", "😊": "emoji", "한국어": "ko_KR"}
language = language_dict[LANGUAGE]
debug = "--debug" if DEBUG else ""

# Start WebUI
!python webUI.py --use_cloud --share --language {language} --model_download_link {MODEL_DOWNLOAD_LINK} {debug}